# Final Project - Part 2

## 1. Title: “CareerChat: Your AI Job Outreach Assistant”

Team:
1.   Anshika Bajpai
2.   Brendan Kelly
3.   Cassie Cagwin

11/9/2025

## 2. Preprocessing 30pts

---

Provide all essential steps that you deem necessary for your application

In [ ]:
# import nbformat

# fname = "/Users/anshikabajpai/Desktop/nlp/project/CareerChat/Base_code.ipynb"
# nb = nbformat.read(fname, as_version=4)

# # Drop corrupted widget metadata if present
# nb.metadata.pop("widgets", None)

# nbformat.write(nb, fname)
# print("✅ Cleaned and ready for Colab!")


✅ Cleaned and ready for Colab!


In [2]:
##Ask if they'd like to upload their resume or fill out a form with their profile
profile_creation_method = 'Yes, I would like to upload my resume.' #@param ['Yes, I would like to upload my resume.', 'No, I would like to fill out the profile form.']
print(f"Selection: {profile_creation_method}")

Selection: Yes, I would like to upload my resume.


In [3]:
##Profile info imports
##Name, work experience, education, skills, career interest
import ipywidgets as widgets
from IPython.display import display, clear_output

if profile_creation_method == 'No, I would like to fill out the profile form.':

  name_input = widgets.Text(
      description='Name:',
      placeholder='Enter your full name',
      layout=widgets.Layout(width='auto')
  )


  email_input = widgets.Text(
      description='Email:',
      placeholder='user@example.com',
      layout=widgets.Layout(width='auto')
  )

  experience_input = widgets.Textarea(
      description='Work Experience:',
      placeholder='List companies, roles, and achievements (one per line)',
      rows=5, # Number of visible rows
      layout=widgets.Layout(width='auto')
  )

  education_input = widgets.Textarea(
      description='Education:',
      placeholder='List degrees, institutions, and dates',
      rows=3,
      layout=widgets.Layout(width='auto')
  )

  skills_input = widgets.Textarea(
      description='Skills:',
      placeholder='List your key skills (e.g., Python, SQL, NLP)',
      rows=3,
      layout=widgets.Layout(width='auto')
  )

  interest_input = widgets.Textarea(
      description='Career Interests:',
      placeholder='Please describe your career interests',
      rows=3,
      layout=widgets.Layout(width='auto')
  )

  submit_button = widgets.Button(description="Submit Profile", button_style='success')
  output_area = widgets.Output()

  def on_submit_click(b):
      with output_area:
          clear_output(wait=True)

          print("Profile Information")
          print(f"Name: {name_input.value}")
          print(f"Email: {email_input.value}")
          print("Career Interests:")
          print(interest_input.value)
          print("Work Experience:")
          print(experience_input.value)
          print("\nEducation:")
          print(education_input.value)
          print("\nSkills:")
          print(skills_input.value)

  submit_button.on_click(on_submit_click)

  form_widgets = [
      name_input,
      email_input,
      widgets.VBox([
          widgets.HTML("<b>Career Interests:</b>"),
          interest_input
      ]),
      widgets.VBox([
          widgets.HTML("<b>Work Experience (Details):</b>"),
          experience_input
      ]),
      widgets.VBox([
          widgets.HTML("<b>Education (Details):</b>"),
          education_input
      ]),
      widgets.VBox([
          widgets.HTML("<b>Skills (Keywords):</b>"),
          skills_input
      ]),
      submit_button,
      output_area
  ]

  full_form = widgets.VBox(form_widgets)
  display(full_form)


In [5]:
# ##Resume upload
# from google.colab import files
# if profile_creation_method == 'Yes, I would like to upload my resume.':
#     print("Please upload your resume.")
#     resume = files.upload()

#     for fn in resume.keys():
#         print('User uploaded file "{name}" with length {length} bytes'.format(
#             name=fn, length=len(resume[fn])))

In [6]:
##Resume parsing
##PDF only for now?



In [ ]:
##Company
company = input("What company are you messaging about? ")

##Role type
role_type = input("What type of role are you messaging about? ")

##Job title
job_title = input("What is the job title you are messaging about? ")

##Job description
job_description = input("What is the job description you are messaging about? ")

In [ ]:
##Message type selection
##Options: LinkedIn connection notes, InMails, Referral requests, Cover letters, Thank-you notes, Recruiter outreach emails
import ipywidgets as widgets
from IPython.display import display

menu_options = [
    'LinkedIn connection notes',
    'InMails',
    'Referral requests',
    'Cover letters',
    'Thank-you notes',
    'Recruiter outreach emails'
]

message_type_dropdown = widgets.Dropdown(
    options=menu_options,
    value= 'LinkedIn connection notes',
    description='Choose the type of message you want to write:',
    disabled=False,
)

display(message_type_dropdown)

selected_message_type = message_type_dropdown.value

Dropdown(description='Choose the type of message you want to write:', options=('LinkedIn connection notes', 'I…

In [ ]:
##Optional context: tone, structure, content
tone_description = input("Please specify any particular tone you want the message to have.")
structure_description = input("Please specify any particular structure you want the message to have.")
content_description = input("Please specify any particular content you want the message to have.")

Please specify any particular tone you want the message to have.\
Please specify any particular structure you want the message to have.
Please specify any particular content you want the message to have.


## 3. Feature Extraction 30pts

---

Implement any existing feature extraction tools and methods (term frequency, word embeddings etc)

## 4. Main Functionality 10pts

---

This is your main task. For example, if you are creating a translation app, your main task is Translation.

In [ ]:
!pip install pygooglenews

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 8.7 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=4b565284acd2934cada9d2cca8aa7acd186be0a0520a101c0a798e01cdadedc3
  Stored in directory: /root/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [ ]:
!pip install newspaper3k

In [ ]:
!pip install lxml_html_clean

In [ ]:
##Scrape news about the company
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pygooglenews import GoogleNews
from newspaper import Article, Config

company_query = company
gn = GoogleNews()

search_results = gn.search(
    company,
    '1m'
)
news_items = search_results['entries']

parsed_articles = []

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36'
config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 30
config.memoize_articles = False

for item in news_items:
    article = Article(item.link, config=config)

    article.download()
    article.parse()
    article.nlp()

    parsed_articles.append({
        'Title': article.title,
        'Published': item.published,
        'Source': item.source.title,
        'Link': item.link,
        'Summary': article.summary,
        'Full_Text': article.text
    })

In [ ]:
!pip install GoogleNews
import time
import random
from datetime import datetime
from dateutil.relativedelta import relativedelta

import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from newspaper import Article, Config
from GoogleNews import GoogleNews # Use the alternative package

# --- Configuration (Keep robust settings) ---
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 30
config.memoize_articles = False # Avoid cache issues

# --- Setup Search Parameters (assuming company is defined) ---
company_query = "Tesla electric vehicles"
end_date_str = datetime.now().strftime('%m/%d/%Y')
start_date_str = (datetime.now() - relativedelta(months=3)).strftime('%m/%d/%Y')

gn = GoogleNews(start=start_date_str, end=end_date_str)
gn.search(company_query)
news_items = gn.results()
parsed_articles = []

print(f"Starting scrape for {len(news_items)} articles. This will take a few minutes...")

# --- Loop with Sleep ---
for i, item in enumerate(news_items):
    article_link = item.get('link')

    try:
        article = Article(article_link, config=config)

        article.download()
        article.parse()
        article.nlp()

        if not article.text and not article.title:
            raise Exception("Article text and title were blank after parsing.")

        parsed_articles.append({
            'Title': article.title,
            'Published': item.get('date', 'N/A'),
            'Source': item.get('media', 'N/A'),
            'Link': article_link,
            'Summary': article.summary,
            'Full_Text': article.text
        })

    except Exception as e:
        parsed_articles.append({
            'Title': item.get('title', 'N/A'),
            'Published': item.get('date', 'N/A'),
            'Source': item.get('media', 'N/A'),
            'Link': article_link,
            'Summary': "Scraping failed.",
            'Full_Text': f"Error during parsing/download: {e}"
        })

    # Add a random pause between 2 and 6 seconds
    # This is CRITICAL for avoiding blocks/throttling
    time_to_wait = random.uniform(2, 6)
    time.sleep(time_to_wait)

    if (i + 1) % 10 == 0:
        print(f"Processed {i + 1} articles. Waiting...")

Starting scrape for 10 articles. This will take a few minutes...
Processed 10 articles. Waiting...


In [ ]:
df = pd.DataFrame(parsed_articles)
df.head()

##Summarize articles
##????

,Title,Published,Source,Link,Summary,Full_Text
0,Tesla Cybercab almost looks production ready i...,3 minutes ago,Teslarati,https://www.teslarati.com/tesla-cybercab-almos...,Scraping failed.,Error during parsing/download: Article `downlo...
1,Xiaomi EV says Oct deliveries again surpass 40...,4 minutes ago,CnEVPost,https://cnevpost.com/2025/11/01/xiaomi-ev-oct-...,Scraping failed.,Error during parsing/download: Article `downlo...
2,MSN,5 minutes ago,MSN,https://www.msn.com/en-us/money/technology/tes...,,
3,"New study of 1,300 EVs shatters one of the big...",1 hour ago,supercarblondie.com,https://supercarblondie.com/new-study-shatters...,Scraping failed.,Error during parsing/download: \n*************...
4,Volkswagen Secures $426M Brazilian Loan to Acc...,3 hours ago,EVXL.co,https://evxl.co/2025/10/31/volkswagen-brazilia...,Scraping failed.,Error during parsing/download: \n*************...


In [ ]:
##Generate message


## 5. Personal Contribution Statement 10pts

---

*   Summary of tasks and team members' contributions
*   Proofreading




**Our assignments for Part 2...**

1. PDF parsing + turn into profile - **Cassie**
	* Offer parsed resume as a form where they can correct values
2. News scraping (+ Reddit https://www.reddit.com/dev/api/) + summarization - **Brendan**
	* Offer filters for topics in news
3. Prompted engineering for messages (including offering news highlights to choose from + maybe reddit) + code to generate message - **Anshika**
	* Who to send to form: Who? How do you know them?
	* Creativity (temperature) measure?

***** Don't forget to put in your specific feedback! *****